In [21]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [22]:
df = pd.read_csv(r"C:/Users/swapn/Desktop/final rs/modified_dataset3.csv")
df.head()


,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,₹399,"₹1,099",64%,4.2,24269,High Compatibility : Compatible With iPhone 12...,AG3D6O4STAQKAY2UVGEUV46KN35Q,Manav,R3HXWT0LRP0NMF,Satisfied,Looks durable Charging is fine tooNo complains,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
1,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,₹399,"₹1,099",64%,4.2,24269,High Compatibility : Compatible With iPhone 12...,AHMY5CWJMMK5BJRBBSNLYT3ONILA,Adarsh gupta,R2AJM3LFTLZHFO,Charging is really fast,Charging is really fast,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
2,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,₹399,"₹1,099",64%,4.2,24269,High Compatibility : Compatible With iPhone 12...,AHCTC6ULH4XB6YHDY6PCH2R772LQ,Sundeep,R6AQJGUP6P86,Value for money,good product.,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
3,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,₹399,"₹1,099",64%,4.2,24269,High Compatibility : Compatible With iPhone 12...,AGYHHIERNXKA6P5T7CZLXKVPT7IQ,S.Sayeed Ahmed,R1KD19VHEDV0OR,Product review,Till now satisfied with the quality.,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
4,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,₹399,"₹1,099",64%,4.2,24269,High Compatibility : Compatible With iPhone 12...,AG4OGOFWXJZTQ2HKYIOCOY3KXF2Q,jaspreet singh,R3C02RMYQMK6FC,Good quality,This is a good product . The charging speed is...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...


In [23]:
df['discounted_price'] = df['discounted_price'].astype(str).str.replace('₹', '').str.replace(',', '').astype(float)
df['actual_price'] = df['actual_price'].astype(str).str.replace('₹', '').str.replace(',', '').astype(float)
df['discount_percentage'] = df['discount_percentage'].astype(str).str.replace('%','').astype(float)/100


In [24]:
# The rating column has a value with an incorrect character, so we will exclude
# the row to obtain a clean dataset.
count = df['rating'].str.contains('\|').sum()
print(f"Total de linhas com '|' na coluna 'rating': {count}")
df = df[df['rating'].apply(lambda x: '|' not in str(x))]
count = df['rating'].str.contains('\|').sum()
print(f"Total de linhas com '|' na coluna 'rating': {count}")

Total de linhas com '|' na coluna 'rating': 8
Total de linhas com '|' na coluna 'rating': 0


In [25]:
df['rating'] = df['rating'].astype(str).str.replace(',', '').astype(float)
df['rating_count'] = df['rating_count'].astype(str).str.replace(',', '').astype(float)

le = LabelEncoder()
df['user_id_encoded'] = le.fit_transform(df['user_id'])

In [26]:
# POPULAR BASED RECOMMENDATION
import pandas as pd


def popular_products():
# Calculate the mean rating across all products
    C = df['rating'].mean()

# Calculate the 90th percentile of the number of ratings
    m = df['rating_count'].quantile(0.9)

    # Filter out movies that have a rating count less than m
    qualified_products = df[df['rating_count'] >= m]

    # Compute the weighted rating for each qualified product
    def weighted_rating(x, m=m, C=C):
        v = x['rating_count']
        R = x['rating']
        return (v / (v + m) * R) + (m / (v + m) * C)

    # Apply the function to the DataFrame
    qualified_products['weighted_rating'] = qualified_products.apply(weighted_rating, axis=1)

# Sort products based on score
    qualified_products = qualified_products.sort_values('weighted_rating', ascending=False)

    # Print the top recommendations
    print(qualified_products[['product_name', 'rating', 'rating_count', 'weighted_rating']])


In [27]:
unique_users = df['user_id_encoded'].unique()
train_users = np.random.choice(unique_users, size=int(0.8 * len(unique_users)), replace=False)
train_df = df[df['user_id_encoded'].isin(train_users)]
test_df = df[~df['user_id_encoded'].isin(train_users)]

user_with_most_interactions = df['user_id_encoded'].value_counts().idxmax()

# Listing all the products bought by this user
products_bought = df[df['user_id_encoded'] == user_with_most_interactions]['product_name'].unique()

print(f"User with most interactions: {user_with_most_interactions}")
print(f"Products bought by {user_with_most_interactions}: {products_bought}")


User with most interactions: 4773
Products bought by 4773: ['Flix Micro Usb Cable For Smartphone (Black)'
 'FLiX (Beetel USB to Micro USB PVC Data Sync & 2A Fast Charging Cable, Made in India, 480Mbps Data Sync, Solid Cable, 1 Meter Long USB Cable for Micro USB Devices (White)(XCD-M11)'
 'Flix (Beetel) Usb To Type C Pvc Data Sync And 2A 480Mbps Data Sync, Tough Fast Charging Long Cable For Usb Type C Devices, Charging Adapter (White, 1 Meter) - Xcd-C12'
 'FLiX (Beetel USB to Type C PVC Data Sync & 15W(3A) TPE Fast Charging Cable, Made in India, 480Mbps Data Sync, 1 Meter Long cable for all Andriod & all Type C Devices (Black)(XCD - FPC02)'
 'FLiX (Beetel Flow USB to Micro USB PVC Data Sync & 12W(2.4A) Fast Charging Cable,Made in India,480Mbps Data Sync,Solid Cable,1 Meter Long cable for all Andriod & Micro USB Devices (Black)(XCD-FPM01)'
 'FLiX (Beetel) USB to iPhone Lightning Textured Pattern Data Sync & 2A Fast Charging Cable, Made in India, 480Mbps Data Sync, Tough Cable, 1 Meter Lo

In [28]:
# one_observation_per_user = df.drop_duplicates(subset='user_id_encoded', keep='first')



In [29]:
# COLLABORATIVE FILTERING
x = train_df.groupby('user_id_encoded').count()['rating'] >3
users_rated = x[x].index




In [30]:

filtered_df = train_df[train_df['user_id_encoded'].isin(users_rated)]

In [31]:
y = filtered_df.groupby('product_name').count()['rating'] >1
high_rated_products = y[y].index

In [32]:
high_rated_products


Index(['Acer 100 cm (40 inches) P Series Full HD Android Smart LED TV AR40AR2841FDFL (Black)',
       'Acer 109 cm (43 inches) I Series 4K Ultra HD Android Smart LED TV AR43AR2851UDFL (Black)',
       'Acer 127 cm (50 inches) I Series 4K Ultra HD Android Smart LED TV AR50AR2851UDFL (Black)',
       'Acer 139 cm (55 inches) I Series 4K Ultra HD Android Smart LED TV AR55AR2851UDFL (Black)',
       'Acer 80 cm (32 inches) I Series HD Ready Android Smart LED TV AR32AR2841HDFL (Black)',
       'Amazon Basics High-Speed HDMI Cable, 6 Feet (2-Pack),Black',
       'Amazon Basics High-Speed HDMI Cable, 6 Feet - Supports Ethernet, 3D, 4K video,Black',
       'AmazonBasics Flexible Premium HDMI Cable (Black, 4K@60Hz, 18Gbps), 3-Foot',
       'AmazonBasics Micro USB Fast Charging Cable for Android Phones with Gold Plated Connectors (3 Feet, Black)',
       'AmazonBasics USB 2.0 - A-Male to A-Female Extension Cable for Personal Computer, Printer (Black, 9.8 Feet/3 Meters)',
       ...
       'iQOO 

In [33]:
final_rating = filtered_df[filtered_df['product_name'].isin(high_rated_products)]

In [34]:
pt = final_rating.pivot_table(index='product_name' , columns='user_id_encoded', values='rating')
pt.fillna(0,inplace=True)
pt


user_id_encoded,11,35,45,78,80,103,179,228,290,368,...,8716,8717,8748,8794,8843,8847,8851,8861,8992,9029
product_name,,,,,,,,,,,,,,,,,,,,,
Acer 100 cm (40 inches) P Series Full HD Android Smart LED TV AR40AR2841FDFL (Black),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Acer 109 cm (43 inches) I Series 4K Ultra HD Android Smart LED TV AR43AR2851UDFL (Black),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Acer 127 cm (50 inches) I Series 4K Ultra HD Android Smart LED TV AR50AR2851UDFL (Black),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Acer 139 cm (55 inches) I Series 4K Ultra HD Android Smart LED TV AR55AR2851UDFL (Black),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Acer 80 cm (32 inches) I Series HD Ready Android Smart LED TV AR32AR2841HDFL (Black),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"pTron Solero M241 2.4A Micro USB Data & Charging Cable, Made in India, 480Mbps Data Sync, Durable 1-Meter Long USB Cable for Micro USB Devices (White)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"pTron Solero MB301 3A Micro USB Data & Charging Cable, Made in India, 480Mbps Data Sync, Strong & Durable 1.5-Meter Nylon Braided USB Cable for Micro USB Devices - (Black)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"pTron Solero T241 2.4A Type-C Data & Charging USB Cable, Made in India, 480Mbps Data Sync, Durable 1-Meter Long USB Cable for Smartphone, Type-C USB Devices (White)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_score = cosine_similarity(pt)

In [36]:
similarity_score[0]

array([1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.])

In [37]:
# def get_recommendations(product_name):
#     """Return a list of recommended products for a given product."""
#     index = np.where(pt.index == product_name)[0][0]
#     similar_items = sorted(list(enumerate(similarity_score[index])), key=lambda x: x[1], reverse=True)[1:7]
    
#     recommended_products = [pt.index[i[0]] for i in similar_items]
#     return recommended_products
    
    

In [38]:

def get_recommendations(product_name):
    """Return a list of recommended products for a given product."""
    try:
        index = np.where(pt.index == product_name)[0][0]
        print(f"Index for {product_name}: {index}")
        
        similar_items = sorted(list(enumerate(similarity_score[index])), key=lambda x: x[1], reverse=True)[1:6]
        print(f"Similar items: {similar_items}")
        
        recommended_products = [pt.index[i[0]] for i in similar_items]
        print(f"Recommended products: {recommended_products}")
    except IndexError:
        print(f"Error: {product_name} not found in index.")
        # print(qualified_products[['product_name', 'rating', 'rating_count', 'weighted_rating']])
        recommended_products = qualified_products[['product_name', 'rating', 'rating_count', 'weighted_rating']]

    return recommended_products

In [39]:
recommended = get_recommendations('FLiX (Beetel USB to Type C PVC Data Sync & 15W(3A) TPE Fast Charging Cable, Made in India, 480Mbps Data Sync, 1 Meter Long cable for all Andriod & all Type C Devices (Black)(XCD - FPC02)')
# For a given product



gd=['FLiX (Beetel USB to Type C PVC Data Sync & 15W(3A) TPE Fast Charging Cable, Made in India, 480Mbps Data Sync, 1 Meter Long cable for all Andriod & all Type C Devices (Black)(XCD - FPC02)', 'FLiX (Beetel Flow USB to Micro USB PVC Data Sync & 12W(2.4A) Fast Charging Cable,Made in India,480Mbps Data Sync,Solid Cable,1 Meter Long cable for all Andriod & Micro USB Devices (Black)(XCD-FPM01)']

Index for FLiX (Beetel USB to Type C PVC Data Sync & 15W(3A) TPE Fast Charging Cable, Made in India, 480Mbps Data Sync, 1 Meter Long cable for all Andriod & all Type C Devices (Black)(XCD - FPC02): 26
Similar items: [(25, 0.9999999999999998), (26, 0.9999999999999998), (27, 0.9999999999999998), (28, 0.9999999999999998), (29, 0.9999999999999998)]
Recommended products: ['FLiX (Beetel USB to Micro USB PVC Data Sync & 2A Fast Charging Cable, Made in India, 480Mbps Data Sync, Solid Cable, 1 Meter Long USB Cable for Micro USB Devices (White)(XCD-M11)', 'FLiX (Beetel USB to Type C PVC Data Sync & 15W(3A) TPE Fast Charging Cable, Made in India, 480Mbps Data Sync, 1 Meter Long cable for all Andriod & all Type C Devices (Black)(XCD - FPC02)', 'FLiX (Beetel) 3in1 (Type C|Micro|Iphone Lightening) Textured Pattern 3A Fast Charging Cable with QC & PD Support for Type C,Micro USB & Lightning Iphone Cable,Made in India,1.5 Meter Long Cable(T101)', 'FLiX (Beetel) USB to Type C PVC Data Sync & 2A Smartph

In [40]:
print("hi")
def precision_recall_at_k(recommended, ground_truth, k=10):
    """Calculate precision and recall at k for a set of recommendations."""
    print(f"Recommended: {recommended}")
    print(f"Ground Truth: {ground_truth}") 
    
    if len(recommended) > k:
        recommended = recommended[:k]

    # Number of relevant and recommended items in top k
    rel_and_rec_k = sum((item in ground_truth) for item in recommended)
    print(f"Relevant and recommended items in top k: {rel_and_rec_k}")
    
    # Precision@K: Proportion of recommended items that are relevant
    precision = rel_and_rec_k / len(recommended) if recommended else 0
    
    # Recall@K: Proportion of relevant items that are recommended
    recall = rel_and_rec_k / len(ground_truth) if ground_truth else 0
    
    print(f"Precision at {k}: {precision:.4f}")
    print(f"Recall at {k}: {recall:.4f}")
    
    return precision, recall

precision_recall_at_k(recommended , gd , k=10)




hi
Recommended: ['FLiX (Beetel USB to Micro USB PVC Data Sync & 2A Fast Charging Cable, Made in India, 480Mbps Data Sync, Solid Cable, 1 Meter Long USB Cable for Micro USB Devices (White)(XCD-M11)', 'FLiX (Beetel USB to Type C PVC Data Sync & 15W(3A) TPE Fast Charging Cable, Made in India, 480Mbps Data Sync, 1 Meter Long cable for all Andriod & all Type C Devices (Black)(XCD - FPC02)', 'FLiX (Beetel) 3in1 (Type C|Micro|Iphone Lightening) Textured Pattern 3A Fast Charging Cable with QC & PD Support for Type C,Micro USB & Lightning Iphone Cable,Made in India,1.5 Meter Long Cable(T101)', 'FLiX (Beetel) USB to Type C PVC Data Sync & 2A Smartphone Fast Charging Cable, Made in India, 480Mbps Data Sync, Tough Cable, 1 Meter Long USB Cable for USB Type C Devices Black XCD-C12', 'FLiX (Beetel) USB to iPhone Lightning Textured Pattern Data Sync & 2A Fast Charging Cable, Made in India, 480Mbps Data Sync, Tough Cable, 1 Meter Long USB Cable for Apple Devices (Black)(XCD-L102)']
Ground Truth: ['FLi

(0.2, 0.5)